In [1]:
import mysql.connector
import pandas as pd

# Connection with the database
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="rootpassword",
  database="recetas_db"
)

# Load data from the database into dataframes
ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
allergens_df = pd.read_sql_query("SELECT * FROM alergenos", conn)
restriccions_df = pd.read_sql_query("SELECT * FROM restricciones", conn)

# Load data from the database into dataframes
conn.close()

C:\Users\jcela\AppData\Local\Temp\ipykernel_9936\2738617342.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_9936\2738617342.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_9936\2738617342.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  allergens_df = pd.read_sql_query

In [2]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass

@dataclass
class Ant:
    recipe: List[int]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solution = None
        self.best_solution_fitness = float('inf')

    def _optimize(self) -> List[int]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if solution_fitness < self.best_solution_fitness:
                    self.best_solution = solution
                    self.best_solution_fitness = solution_fitness
            self.update_pheromone_matrix(solutions)
        return self.best_solution
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set() # to avoid repeating ingredients
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient)
                else:
                    break  # No more valid ingredients
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[int], selected_ingredients: set) -> int:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]
        
        for ingredient in available_ingredients:
            if self._check_ingredient(ingredient):
                pheromone = sum(self.pheromone_matrix[ingredient - 1][i - 1] for i in current_recipe)
                desirability = self._calculate_desirability(ingredient)
                probabilities.append((ingredient, (pheromone ** self.alpha) * (desirability ** self.beta)))
        
        if not probabilities:
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            return None
        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        selected_ingredient = random.choices([ing for ing, _ in probabilities], [prob for _, prob in probabilities])[0]
        return selected_ingredient
    
    def _calculate_desirability(self, ingredient_id: int) -> float:
        # For simplicity, let's assume all ingredients have equal desirability
        # You can replace this with a more sophisticated desirability calculation
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        '''
            constraints format: vegan, vegetarian, allergen list
            1,1,10,11,1,2,3 -> vegan, vegetarian and allergic to allergens with ids 10,11,1,2,3
            0.1 -> vegetarian, not allegic
            0.0 -> not vegan, not vegetarian, not allergic
            0,0,10,2 -> non-vegan, non-vegetarian and allergic to allergens with ids 10 and 2
            1,1,10,2 -> vegan, vegetarian and is allergic to allergens with id 10 and 2
        '''
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True

    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'].isin(solution.recipe)].sum()
        # Implement your fitness function based on total_nutrition
        # For now, let's assume lower calories is better
        return total_nutrition['Calorias']
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            pheromone_contribution = self.pheromone_deposit / self.evaluate_solution_fitness(ant)
            for i in range(len(ant.recipe) - 1):
                self.pheromone_matrix[ant.recipe[i] -1][ant.recipe[i+1] - 1] += pheromone_contribution

# Example usage
if __name__ == "__main__":
    #hard_constraints = '1,1,10,11,1,2,3'

    ant_colony = AntColony(ingredients_df, recipes_ingredients_df, allergens_df, restriccions_df, '0,0,1')
    best_recipe = ant_colony._optimize()
    print(best_recipe)


C:\Users\jcela\AppData\Local\Temp\ipykernel_9936\2793784619.py:133: RuntimeWarning: divide by zero encountered in scalar divide
  pheromone_contribution = self.pheromone_deposit / self.evaluate_solution_fitness(ant)


Ant(recipe=[])
